In [1]:
import json
from datetime import datetime
import pandas as pd 
from tqdm import tqdm

import numpy as np

In [3]:
file = "../data/Beauty_and_Personal_Care.jsonl"
data = []

with open(file, 'r') as fp:
    for line in tqdm(fp):
        
        dat = json.loads(line.strip())
        if datetime.fromtimestamp(dat["timestamp"] / 1000).year >= 2019:
            data.append({key: dat[key] for key in ["user_id", "asin", "parent_asin", "rating", "timestamp"]})

0it [00:00, ?it/s]

23911390it [01:15, 316416.90it/s]


In [4]:
product_data = pd.DataFrame(data)
product_data["year"] = product_data["timestamp"].apply(lambda x: datetime.fromtimestamp(x / 1000).year)

product_data.to_parquet("Beauty_and_Personal_Care.parquet")

In [5]:
product_data["year"].value_counts()

year
2021    3723646
2022    3684304
2020    3551569
2019    2585809
2023    1677485
Name: count, dtype: int64

# Load Meta Data

In [ ]:
file = "../data/meta_Beauty_and_Personal_Care.jsonl"
data = []

parent_asin = product_data["parent_asin"].unique().tolist()

with open(file, 'r') as fp:
    for line in tqdm(fp):
        
        dat = json.loads(line.strip())

        if dat["parent_asin"] in parent_asin:
            data.append(dat)

128193it [27:33, 72.66it/s] 

In [ ]:
meta = pd.DataFrame(data)

In [ ]:
def format_price(price):

    if type(price) == str and "from" in price:
        return float(price.replace("from", "").strip())
    elif type(price) == str:
        return np.nan
    else:
        return price

In [ ]:
meta["price"] = meta["price"].apply(lambda x: format_price(x))

year
2021    1881733
2022    1667588
2023     651447
Name: count, dtype: int64

In [ ]:
meta.to_parquet("meta_Beauty_and_Personal_Care.parquet")